# 1D Fourier Transform - part 5

<a id='ref'></a>
## References

* Van Loan, C. F., (1992). Computational Frameworks for the Fast Fourier Transform, Frontiers in applied mathematics : 10, SIAM, ISBN 0-89871-285-8.

* Golub, G. H. and C. F. Van Loan, (2013), Matrix computations, 4th edition, Johns Hopkins University Press, ISBN 978-1-4214-0794-4.

In [1]:
import numpy as np
from scipy.linalg import dft

<a id='sec1'></a>
## Decomposition of Fourier matrix into even-odd columns

Let's remember the DFT (equation 51 of notebook `fourier_1D_4`) and IDFT (equation 52 of notebook `fourier_1D_4`) in matrix notation, for a particular scale factor combination:

<a id='eq1'></a>

$$
\mathbf{G} = \mathbf{F}_{N} \, \mathbf{g} \tag{1}
$$

and

<a id='eq2'></a>

$$
\mathbf{g} = \frac{1}{N} \, \mathbf{F}^{H}_{N} \, \mathbf{G} \quad, \tag{2}
$$

where $\mathbf{F}_{N}$ is an $N \times N$ complex matrix called *Discrete Fourier Transform Matrix* or simply *DFT matrix*, with element $nk$ given by 

<a id='eq3'></a>

$$
\left( w_{N} \right)^{nk} = \left( e^{\, -i \, 2 \, \pi \, \mathbin{/} \, N} \right)^{nk} \: . \tag{3}
$$

It can be shown that:

* $\mathbf{F}^{\ast}_{N} = \mathbf{F}^{H}_{N}$

* $\mathbf{F}^{-1}_{N} = \frac{1}{N} \, \mathbf{F}^{\ast}_{N}$

* $N \, \mathbf{I} = \mathbf{F}^{H}_{N} \mathbf{F}_{N}$

where $\mathbf{F}^{\ast}_{N}$ is the complex conjugate without transposition and $\mathbf{F}^{H}_{N}$ is the conjugate transpose.

For the particular case in which $N$ is even, lets defined a new matrix:

<a id='eq4'></a>

$$
\breve{\mathbf{F}}_{N} = 
\mathbf{F}_{N} \, \mathbf{P}^{\top} = 
\begin{bmatrix}
\mathbf{F}_{N}^{e} & \mathbf{F}_{N}^{o}
\end{bmatrix} \quad , \tag{4}
$$

where $\mathbf{P}$ is a permutation matrix, $\mathbf{F}_{N}^{e}$ is an $N \times N/2$ matrix containing the even columns (e.g., $0, 2, \dots, N-2$) of $\mathbf{F}_{N}$ and $\mathbf{F}_{N}^{o}$ is an $N \times N/2$ matrix containing the odd columns (e.g., $1, 3, \dots, N-1$) of $\mathbf{F}_{N}$. The permutation matrix $\mathbf{P}$ for the particular case in which $N = 6$ is given by:

$$
\mathbf{P} = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1
\end{bmatrix} \quad .
$$

In [2]:
N = 6
P = np.identity(N)[[0,2,4,1,3,5]]
print(P)

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]


In [3]:
print(P.T)

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]


In [4]:
np.allclose(np.dot(P.T, P), np.identity(N))

True

In [5]:
np.allclose(np.dot(P, P.T), np.identity(N))

True

In [6]:
np.allclose(np.dot(P, P), np.identity(N))

False

In [7]:
A = np.reshape(np.arange(N*N), (N,N))
print(A)

[[ 0  1  2  3  4  5]
 [ 6  7  8  9 10 11]
 [12 13 14 15 16 17]
 [18 19 20 21 22 23]
 [24 25 26 27 28 29]
 [30 31 32 33 34 35]]


In [8]:
print(np.dot(A, P.T))

[[ 0.  2.  4.  1.  3.  5.]
 [ 6.  8. 10.  7.  9. 11.]
 [12. 14. 16. 13. 15. 17.]
 [18. 20. 22. 19. 21. 23.]
 [24. 26. 28. 25. 27. 29.]
 [30. 32. 34. 31. 33. 35.]]


It can be shown that $\mathbf{F}_{N}^{e}$ and $\mathbf{F}_{N}^{o}$ ([equation 4](#eq4)) can be rewritten as follows:

<a id='eq5'></a>

$$
\mathbf{F}_{N}^{e} = \begin{bmatrix}
\mathbf{F}_{N/2} \\ \mathbf{F}_{N/2}
\end{bmatrix} \tag{5}
$$

and

<a id='eq6'></a>

$$
\mathbf{F}_{N}^{o} = \begin{bmatrix}
\boldsymbol{\Omega}_{N/2} \, \mathbf{F}_{N/2} \\ 
-\boldsymbol{\Omega}_{N/2} \, \mathbf{F}_{N/2}
\end{bmatrix} \quad , \tag{6}
$$

where $\mathbf{F}_{N/2}$ is the $N/2 \times N/2$ Fourier matrix and $\boldsymbol{\Omega}_{N/2}$ is an $N/2 \times N/2$ diagonal matrix with diagonal elements defined by the  following vector:

<a id='eq7'></a>

$$
\boldsymbol{\omega}_{N} = \begin{bmatrix}
w_{N}^{0} \\
w_{N}^{1} \\
w_{N}^{2} \\
\vdots \\
w_{N}^{N/2-1} \\
\end{bmatrix} \quad , \tag{7}
$$

with $w_{N}$ defined by [equation 3](#eq3) .

By using equations [5](#eq5)-[7](#eq7), we can rewrite the matrix $\breve{\mathbf{F}}_{N}$ ([equation 4](#eq4)) as follows:

<a id='eq8'></a>

$$
\breve{\mathbf{F}}_{N} = \begin{bmatrix} \begin{split}
\mathbf{F}_{N/2} \:\: & &\boldsymbol{\Omega}_{N/2} \, \mathbf{F}_{N/2} \\
\mathbf{F}_{N/2} \:\: & -&\boldsymbol{\Omega}_{N/2} \, \mathbf{F}_{N/2}
\end{split} \end{bmatrix} \quad . \tag{8}
$$

Finally, all this blablabla shows that:

$$
\begin{split}
\mathbf{F}_{N} \, \mathbf{x} 
&= \overbrace{\mathbf{F}_{N} \, \mathbf{P}^{\top}}^{\breve{\mathbf{F}}_{N}} \, 
\mathbf{P} \, \mathbf{x} \\
&= \begin{bmatrix} 
\mathbf{F}_{N/2} & \boldsymbol{\Omega}_{N/2} \, \mathbf{F}_{N/2} \\
\mathbf{F}_{N/2} & -\boldsymbol{\Omega}_{N/2} \, \mathbf{F}_{N/2}
\end{bmatrix} 
\begin{bmatrix}
\mathbf{x}^{e} \\ \mathbf{x}^{o}
\end{bmatrix} \\
&= \begin{bmatrix} 
\mathbf{I}_{N/2} & \boldsymbol{\Omega}_{N/2} \\
\mathbf{I}_{N/2} & -\boldsymbol{\Omega}_{N/2} 
\end{bmatrix} 
\begin{bmatrix}
\mathbf{F}_{N/2} \, \mathbf{x}^{e} \\ 
\mathbf{F}_{N/2} \, \mathbf{x}^{o}
\end{bmatrix} 
\end{split}
$$

$$
\begin{bmatrix} 
\mathbf{y}^{e} \\
\mathbf{y}^{o} 
\end{bmatrix} = 
\begin{bmatrix}
\left( \mathbf{F}_{N/2} \, \mathbf{x}^{e} \right) 
+ \boldsymbol{\omega}_{N} \circ \left( \mathbf{F}_{N/2} \, \mathbf{x}^{o} \right) \\ 
\left( \mathbf{F}_{N/2} \, \mathbf{x}^{e} \right) 
- \boldsymbol{\omega}_{N} \circ \left( \mathbf{F}_{N/2} \, \mathbf{x}^{o} \right)
\end{bmatrix}
$$

In [10]:
N = 20

In [11]:
FN = dft(N, scale=None)

In [12]:
w_N = np.exp(-1j*2*np.pi/N)

In [13]:
omega_N = np.power(w_N, np.arange(N//2))

In [14]:
Omega_N = np.diag(omega_N)

In [15]:
np.arange(N//2)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [16]:
FN_half = dft(N//2, scale=None)

In [17]:
FN_even = np.vstack([FN_half, 
                     FN_half])

In [18]:
FN_odd = np.vstack([np.dot( Omega_N, FN_half), 
                    np.dot(-Omega_N, FN_half)])

In [19]:
np.allclose(FN[:,0::2],FN_even)

True

In [ ]:
np.allclose(FN[:,1::2],FN_odd)